In [22]:
import wget

In [23]:
# url = "https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py"
# wget.download(url)

In [24]:
#Docoment.json download
# url="https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/refs/heads/main/01-intro/documents.json"
# wget.download(url)

In [25]:
import minsearch

In [4]:
import json

In [27]:
with open('documents.json','rt') as f_in:
    docs_raw=json.load(f_in)

In [28]:
documents=[]

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course']=course_dict['course']
        documents.append(doc)


Index the Document

In [29]:
index =minsearch.Index(
    text_fields=["question","text","section"],
    keyword_fields=["course"]

)

In [30]:
index.fit(documents)


In [31]:
#q='the course has already started, can i still enroll?'

In [32]:
from openai import OpenAI

In [47]:
from dotenv import load_dotenv
import os
from openai import OpenAI

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")




In [48]:
client = OpenAI(api_key=api_key)

In [35]:
def search(query):
    boost={'question':3.0,'section':0.5}

    results=index.search(
        query=query,
        filter_dict={'course':'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
)
    return results

In [30]:
def build_prompt(query,search_results):
    prompt_template="""
You're a course teaching assistance. Answer the QUESTION based on the CONTEXT from the FAQ databalse. Use only the facts from the CONTEXT when answering the QUESTION.


Question :{question}

CONTEXT:
{context}
""".strip()


    context = ""
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"


    prompt=prompt_template.format(question=query,context=context).strip()
    return prompt

In [31]:
def llm(prompt):
    response=client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role":"user","content":prompt}]
)
    return response.choices[0].message.content

In [38]:
def rag(query):
    search_results=search(query)
    prompt=build_prompt(query,search_results)
    answer=llm(prompt)
    return answer

In [39]:
rag("the course has already started. Can i still enroll?")

"Yes, you can still enroll in the course even after it has started. However, be mindful of the deadlines for turning in the final projects. It's advisable not to leave everything until the last minute."


Pull the latest elastic search

docker run -it --rm --name elasticsearch -p 9200:9200 -p 9300:9300 -e "discovery.type=single-node" -e "xpack.security.enabled=false" elasticsearch:8.17.6


after run this :

curl http://localhost:9200
and get the hashsvalue.

In [104]:
import json
from elasticsearch import Elasticsearch

In [105]:
with open('documents.json','rt') as f_in:
    docs_raw=json.load(f_in)

In [106]:
documents=[]

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course']=course_dict['course']
        documents.append(doc)


In [87]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [107]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")
print(es.info())


{'name': '41ed08ed9fe8', 'cluster_name': 'docker-cluster', 'cluster_uuid': '98JYwtj2SbKFOS3Z3hczaQ', 'version': {'number': '8.17.6', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'dbcbbbd0bc4924cfeb28929dc05d82d662c527b7', 'build_date': '2025-04-30T14:07:12.231372970Z', 'build_snapshot': False, 'lucene_version': '9.12.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


Index Settings

In [108]:
index_settings={
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name="course-questions"
es_client.indices.create(index=index_name,body=index_settings)

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [course-questions/NzjvC16dRG2tw0CitelN-Q] already exists')

In [109]:
for doc in documents:
    es_client.index(index=index_name,document=doc)

Set function with _source metric

In [110]:
def elastic_search (query):
    search_query={
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                # "filter": {
                #     "term": {
                #         "course": "data-engineering-zoomcamp"
                #     }
                # }
            }
        }
    }
    response=es_client.search(index=index_name,body=search_query)

    results = []
    for hit in response['hits']['hits']:
        results.append({
            'score': hit['_score'],
            'source': hit['_source']
        })
    return results

In [111]:
query="How do execute a command on a Kubernetes pod?"

In [113]:
print(len(search_results))  # Should be ≤ 5 (if size=5)


NameError: name 'search_results' is not defined

In [112]:
elastic_search(query)

[{'score': 44.56891,
  'source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'}},
 {'score': 44.56891,
  'source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'co

In [95]:
from openai import OpenAI

In [96]:
from dotenv import load_dotenv
import os
from openai import OpenAI

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)


In [65]:
def build_prompt(query,search_results):
    prompt_template="""
You're a course teaching assistance. Answer the QUESTION based on the CONTEXT from the FAQ database. Use only the facts from the CONTEXT when answering the QUESTION.


Question :{question}

CONTEXT:
{context}
""".strip()


    context = ""
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"


    prompt=prompt_template.format(question=query,context=context).strip()
    return prompt

In [66]:
def llm(prompt):
    response=client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role":"user","content":prompt}]
)
    return response.choices[0].message.content

In [73]:
def rag(query):
    search_results=elastic_search(query)
    prompt=build_prompt(query,search_results)
    answer=llm(prompt)
    return answer

In [83]:
elastic_search(query)

[{'score': 44.50556,
  'source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'}},
 {'score': 35.433445,
  'source': {'text': 'Deploy and Access the Kubernetes Dashboard\nLuke',
   'section': '10. Kubernetes and TensorFlow Serving',
   'question': 'Kubernetes-dashboard',
   'course': 'machine-learning-zoomcamp'}},
 {'score': 33.70974,
  'source': {'text': 'You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nIn the Dockerfile, you can provide the folder containing the files that you want to co

In [79]:
results = elastic_search('How do execute a command on a Kubernetes pod?')
for r in results:
    print(f"Score: {r['_score']}")
    print(f"Doc: {r['source']}")
    print("-----------")


KeyError: '_score'

In [68]:
query='How do execute a command on a Kubernetes pod?'

In [69]:
rag(query)

"To execute a command on a Kubernetes pod, the provided CONTEXT does not directly address this specific question. However, based on typical Kubernetes operations and similar practices in Docker, you can use the `kubectl exec` command. Here's how you would generally do it:\n\n```sh\nkubectl exec -it <pod-name> -- <command>\n```\n\nReplace `<pod-name>` with the name of the pod you want to access, and `<command>` with the command you want to execute inside the pod. If you would like an interactive shell, you might use:\n\n```sh\nkubectl exec -it <pod-name> -- /bin/bash\n```\n\nPlease note that this is common Kubernetes usage and not directly from the provided CONTEXT."

In [ ]:
response = es_client.search(index=index_name, body=search_query)
top_score = response['hits']['hits'][0]['_score']
print("Top score:", top_score)



NameError: name 'search_query' is not defined